In [33]:
# 필요한 라이브러리 import 하기

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing
import seaborn as sns 
import optuna 
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score
from sklearn.model_selection import cross_val_score, train_test_split
import os 
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier

In [12]:
# Train data test data import 하기

train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/breastcancer_train_data.csv')
test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/breastcancer_test_data.csv')

In [13]:
train_data.head(5)

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,50,2,1,1,0,1,3,1,1,40,1,1,13,1,50,0
1,59,2,1,0,0,0,1,3,1,12,1,1,11,2,99,0
2,44,2,1,0,0,0,0,2,1,11,1,1,8,2,77,0
3,59,2,0,1,1,2,0,2,1,25,1,0,32,5,107,0
4,69,2,1,0,0,0,3,1,1,18,1,1,4,1,48,0


In [14]:
test_data.head(5)

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,2,1,1,1,2,3,1,1,42,1,1,25,5,48,0
1,62,2,1,2,0,2,0,2,1,70,1,0,3,3,6,1
2,57,2,1,1,1,2,0,2,1,22,1,1,16,5,57,0
3,41,2,1,1,0,1,1,3,1,28,1,1,8,1,103,0
4,65,2,0,1,0,1,0,2,1,46,1,1,3,1,84,0


In [15]:
train_stage_features=['A Stage','differentiate','Race','T Stage ','Grade','Estrogen Status','Progesterone Status','Marital Status','N Stage',
                      '6th Stage','Regional Node Examined','Tumor Size','Reginol Node Positive','Age','Survival Months']

In [16]:
test_stage_features=['A Stage','differentiate','Race','T Stage ','Grade','Estrogen Status','Progesterone Status','Marital Status','N Stage']

In [17]:
## Train data와 Test data의 독립변수와 종속변수 설정하기

train_X=train_data.loc[:,train_stage_features]
train_y=train_data['Status']
new_test_data_X=test_data.loc[:,test_stage_features]
new_test_data_y=test_data['Status']

In [18]:
train_y.value_counts()

0    2726
1     493
Name: Status, dtype: int64

In [19]:
# 데이터셋에 Class weight해주기

ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [20]:
# Train에만 있고 Test data에는 없는 변수들을 하나의 주성분으로 차원축소하기 onlytrain_X 데이터프레임을 만들기

onlytrain_X=train_X.loc[:,['Regional Node Examined','Tumor Size','Reginol Node Positive','Age','Survival Months','6th Stage']]

In [21]:
# 차원축소하기 전에 Scaling을 해주기

onlytrain_X_scaled=StandardScaler().fit_transform(onlytrain_X)

In [1]:
# 차원축소하기 위해 PCA 함수를 import하고 n_components는 1로 지정해주기( 하나의 주성분으로 축소하기 위해서 1로 지정함 )

from sklearn.decomposition import PCA 
pca=PCA(n_components=1)

In [24]:

pca.fit(onlytrain_X_scaled)

PCA(n_components=1)

In [25]:
onlytrain_X_scaled_pca=pca.transform(onlytrain_X_scaled)

In [26]:
pca_columns=['pca_component1']
onlytrain_pca=pd.DataFrame(onlytrain_X_scaled_pca,columns=pca_columns)

In [29]:
# Train에는 있고 Test data에 없는 변수들이 하나의 주성분으로 차원축소됨

onlytrain_pca

,pca_component1
0,-0.211608
1,-1.573696
2,-1.509818
3,0.842010
4,-1.564126
...,...
3214,3.109081
3215,0.939548
3216,-1.126854
3217,0.254370


In [27]:
# Train_X와 Test_X 두 데이터 Feature로 이루어진 Train_X 만들어주기 

train_X=train_data[test_stage_features] 

In [28]:
# Train_X와 이전에 축소한 하나의 주성분 피처를 합쳐서 새로운 데이터 프레임 만들기

new_train_X=pd.concat([train_X,onlytrain_pca],axis=1)


### 베이지안 최적화해주기 - LightgbmRegressor

**1. 5-fold 교차검증 이용해서 Train data로 Validation set을 RMSE가 최저였을 때의 하이퍼파라미터 구하기**

**2. Learning rate 0.01~0.1, max_depth 3~9, n_estimators 100~1000이었을 때에서 가장 최적의 하이퍼파라미터 구하기**

**3. Test data에 새로운 PCA를 하나를 생성해준다.**

In [30]:
import optuna
from lightgbm import LGBMRegressor

# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X, onlytrain_pca['pca_component1'],
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=50)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)

[I 2023-04-17 22:46:13,636] A new study created in memory with name: no-name-7377082c-7280-45d4-a6c9-1110115c8adc
/tmp/ipykernel_226673/772960702.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-17 22:46:16,555] Trial 0 finished with value: 0.35375158210348856 and parameters: {'n_estimators': 934, 'max_depth': 4, 'learning_rate': 0.05029248078282402}. Best is trial 0 with value: 0.35375158210348856.
/tmp/ipykernel_226673/772960702.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-

{'n_estimators': 140, 'max_depth': 3, 'learning_rate': 0.028782666939552757}


#### **Test data 새로운 컬럼 생성하기**

In [31]:
best_lgbmreg=LGBMRegressor(learning_rate=0.028782666939552757,max_depth=3,n_estimators=140,random_state=42)
best_lgbmreg.fit(train_X, onlytrain_pca['pca_component1'])
new_test_data_X['pca_component1']=best_lgbmreg.predict(new_test_data_X[test_stage_features])

## Test data에 없는 변수들을 다 생성했으므로 LightgbmClassifier 분류모델을 만듬

### 베이지안 최적화해주기 - LightgbmClassifier

**1. 5-fold 교차검증 이용해서 Train data로 Validation set을 Auc가 최고였을 때의 하이퍼파라미터 구하기**

**2. Learning rate 0.01~0.1, max_depth 3~9, n_estimators 100~1000이었을 때에서 가장 최적의 하이퍼파라미터 구하기**


In [32]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,new_train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-17 22:55:21,321] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_226673/14971935.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-17 22:55:23,023] Trial 0 finished with value: 0.7247942911900972 and parameters: {'learning_rate': 0.013781302152200506, 'max_depth': 4, 'n_estimators': 397}. Best is trial 0 with value: 0.7247942911900972.
/tmp/ipykernel_226673/14971935.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-17 22:55:28,486] Trial 1 finished with

Best hyperparameters: {'learning_rate': 0.014515862171734463, 'max_depth': 3, 'n_estimators': 509}
Best AUC: 0.7344


In [34]:

model_lgbm=LGBMClassifier(learning_rate= 0.014515862171734463,max_depth=3, n_estimators=509, scale_pos_weight=ratio,random_state=42)
model_lgbm.fit(new_train_X,train_y)

LGBMClassifier(learning_rate=0.014515862171734463, max_depth=3,
               n_estimators=509, random_state=42,
               scale_pos_weight=5.529411764705882)

In [38]:
newtest_pred=model_lgbm.predict_proba(new_test_data_X)[:,1]

In [39]:
# 모델 성능을 평가할 수 있는 함수 만들어주기

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.50,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [40]:
get_clf_prob(new_test_data_y,newtest_pred)

임계값:  0.5
오차행렬
[[513 169]
 [ 59  64]]
정확도: 0.7168, 정밀도: 0.2747, 재현율: 0.5203, F1:0.5889, AUC: 0.6862
